In [7]:
import sys
sys.path.append("C:/MYLOCALFILES/YOLO/models/research/")
sys.path.append("C:/MYLOCALFILES/YOLO/models/research/object_detection")

In [8]:
import numpy as np
import os
import six.moves.urllib as urllib
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from object_detection.utils import ops as utils_ops
%matplotlib inline

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

<h3>Object detection imports</h3>

In [9]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

c:\users\mjbol\anaconda3\envs\pabase\lib\site-packages\matplotlib\__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


<h3>Download Pretrained Models</h3>

In [22]:
#base path where we will save our models
PATH_TO_OBJ_DETECTION = 'C:/MYLOCALFILES/yolo/models/research/object_detection'

# Specify Model To Download. Obtain the model name from the object detection model zoo.
MODEL_NAME = 'ssd_inception_v2_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

In [23]:
DESTINATION_MODEL_TAR_PATH = PATH_TO_OBJ_DETECTION+'/data/'+MODEL_FILE
#opener = urllib.request.URLopener()
#opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, DESTINATION_MODEL_PATH)
tar_file = tarfile.open(DESTINATION_MODEL_TAR_PATH)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, PATH_TO_OBJ_DETECTION+'/data')

<h3>Load Pre-Trained Model</h3>

In [24]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

<h3>Load the Label Map</h3>

In [29]:
PATH_TO_OBJ_DETECTION+'/data/mscoco_label_map.pbtxt'

'C:/MYLOCALFILES/yolo/models/research/object_detection/data/mscoco_label_map.pbtxt'

In [30]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = PATH_TO_OBJ_DETECTION+'/data/mscoco_label_map.pbtxt'
NUM_CLASSES=90

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)